### Test case LVV-T2900 - Verify implementation of maximum time to retrieve single-CCD, single visit PVI image
DMS-REQ-0374-V-01: Max time to retrieve single-CCD coadd cutout image<br>
DMS-REQ-0374-V-02: Satisfies the additional constraint on the number of simultaneous users. 


### Specification: 
A Processed Visit Image of a single CCD shall be retrievable using the VO SIAv2
protocol within pviRetrievalTime with pviRetrievalUsers simultaneous requests for distinct
single-CCD PVIs. PVIs of a single CCD image shall be retrievable in a maximum time of pviRetrievalTime = 10 seconds.

### Requirement Parameter
pviRetrievalTime = 10[second] Maximum time allowed for retrieving a PVI image of a single CCD from a single visit <br>
pviRetrievalUsers = 20[integer] Minimum number of simultaneous users retrieving a single PVI image.

In [1]:
import math

from lsst.rsp.utils import get_pyvo_auth
from lsst.rsp.service import get_siav2_service

from lsst.afw.image import ExposureF 
import lsst.afw.display as afwDisplay
import matplotlib.pyplot as plt


from pyvo.dal.adhoc import DatalinkResults
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord

import time

In [2]:
afwDisplay.setDefaultBackend('matplotlib')
plt.style.use('tableau-colorblind10')
%matplotlib inline

### Requirement Specs

In [3]:
# Maximum time allowed for retrieving a CCD-sized coadd cutout., 
pviRetrievalTime = 10  # seconds

# Minimum number of simultaneous users retrieving a single CCD-sized coadd cutout.
pviRetrievalUsers = 20 

In [4]:
# CCD-sized cutout  
pixels = 4000          #  4kx4k pixels / CCD
pixel_scale_arcsec = 0.2  # arcsec/pixel
pixel_scale_deg = pixel_scale_arcsec / 3600 

# CCD size in degrees
ccd_size_deg = pixels * pixel_scale_deg 
ccd_area_deg2 = ccd_size_deg ** 2
circle_radius_deg = math.sqrt(ccd_area_deg2 / math.pi)

print(f"CCD angular size: {ccd_size_deg:.2f}° × {ccd_size_deg:.2f}°")
print(f"CCD area: {ccd_area_deg2:.2f} deg²")
print(f"Equivalent circular radius: {circle_radius_deg:.6f}°")

NameError: name 'math' is not defined

### Setup services and functions

In [ ]:
sia_service = get_siav2_service("dp1")
assert sia_service is not None

### Generate an image cutout 
Use a DP1 r band image in the ECDFS field for a visit image that was obtained between MJD 60623.256 and 60623.259.

In [ ]:
# Sample region
ra_center = 53.0142245373055
dec_center = -27.91704870900402
lsst_band = 'r'
lsst_visit = 2024120500122
lsst_detector = 5

coords = SkyCoord(ra_center, dec_center, 
                  unit=(u.deg, u.deg))

t1 = Time("60650.23384915526", format='mjd')
t2 = Time("60650.23420150463", format='mjd')

In [ ]:
results = sia_service.search(
    pos=(coords, 0.1*u.deg),
    calib_level=2,
    time=[t1, t2]
).to_table()

In [ ]:
datalink_url = results[0].get('access_url')
dl_result = DatalinkResults.from_result_url(datalink_url,
                                            session=get_pyvo_auth())
image_url = dl_result.getrecord(0).get('access_url')
print(f"Image URL: {image_url}")

### Record the time to retrieve one single-CCD, single visit PVI image via image services
This is the code that needs to be scale tested for ccdRetrievalUsers (20) simultaneous users

In [ ]:
# This is the cell that needs to be benchmarked  for ccdRetrievalUsers (20) simultaneous users
start = time.time()
visit_image = ExposureF(image_url)
end = time.time()

In [ ]:
elapsed = end - start
print(f"Elapsed time: {elapsed:.4f} seconds")
assert elapsed < pviRetrievalTime

In [ ]:
# Plot image 
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(visit_image.image)
plt.show()